In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (
    RidgeClassifier,
    SGDClassifier,
    LogisticRegression,
)
from sklearn.svm import SVC
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
)

In [2]:
df_train = pd.read_csv('../Datasets/loan-prediction/train.csv')
df_train = df_train.drop('Loan_ID', axis=1)
df_train['label'] = 'train'
df_test = pd.read_csv('../Datasets/loan-prediction/test.csv')
df_test = df_test.drop('Loan_ID', axis=1)
df_test['label'] = 'test'
concat_df = pd.concat([df_train , df_test])
cols = set(df_train.columns.tolist())
cols = cols.intersection(df_test.columns.tolist())
cols = list(cols)
# concat_df = concat_df.apply(lambda x:x.fillna(x.value_counts().index[0]))
# concat_df['new_col'] = concat_df['CoapplicantIncome']/concat_df['ApplicantIncome']  
# concat_df['new_col_2'] = concat_df['LoanAmount']*concat_df['Loan_Amount_Term']
# concat_df['new_col_2'] = np.log(concat_df['new_col_2'])
# concat_df['new_col'] = [x if x==0 else 1 for x in concat_df['new_col']]
# threshold = 0.1  # this number is hyper parameter , as much as you reduce it, as much as you remove more points
#                  # you can just try different values the deafult value is (1.5) it works good for most cases
#                  # but be careful, you don't want to try a small number because you may loss some important information from the data .
#                  # that's why I was surprised when 0.1 gived me the best result
            
# new_col_2_out = concat_df['new_col_2']
# q25, q75 = np.percentile(new_col_2_out, 25), np.percentile(new_col_2_out, 75) # Q25, Q75
# print('Quartile 25: {} , Quartile 75: {}'.format(q25, q75))

# iqr = q75 - q25
# print('iqr: {}'.format(iqr))

# cut = iqr * threshold
# lower, upper = q25 - cut, q75 + cut
# print('Cut Off: {}'.format(cut))
# print('Lower: {}'.format(lower))
# print('Upper: {}'.format(upper))

# outliers = [x for x in new_col_2_out if x < lower or x > upper]
# concat_df = concat_df.drop(['CoapplicantIncome', 'ApplicantIncome', 'Loan_Amount_Term', 'LoanAmount'], axis=1)
df = pd.get_dummies(concat_df)
do_pca = False

In [3]:
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y,label_test,label_train
0,5849,0.0,NaN,360.0,1.0,0,1,1,0,1,...,0,1,0,0,0,1,0,1,0,1
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,0,1,0,1,0,0,1,0,0,1
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,0,0,1,0,0,1,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,1,1,0,0,0,1,0,1,0,1
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,0,1,0,0,0,1,0,1,0,1


In [4]:
train = df[df['label_train'] == 1]
train = train.drop('label_train', axis=1)
train = train.drop('label_test', axis=1)
train = train.drop('Loan_Status_N', axis=1)
train = train.drop('Loan_Status_Y', axis=1)
train['Loan_Status'] = df_train['Loan_Status']
train = train.dropna()
#train = train.drop(train[(train['new_col_2'] > upper) | (train['new_col_2'] < lower)].index)
train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
1,4583,1508.0,128.0,360.0,1.0,0,1,0,1,0,...,0,0,1,0,1,0,1,0,0,N
2,3000,0.0,66.0,360.0,1.0,0,1,0,1,1,...,0,0,1,0,0,1,0,0,1,Y
3,2583,2358.0,120.0,360.0,1.0,0,1,0,1,1,...,0,0,0,1,1,0,0,0,1,Y
4,6000,0.0,141.0,360.0,1.0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,1,Y
5,5417,4196.0,267.0,360.0,1.0,0,1,0,1,0,...,1,0,1,0,0,1,0,0,1,Y


In [5]:
test = df[df['label_test'] == 1]
test = test.drop('label_test', axis=1)
test = test.drop('label_train', axis=1)
test = test.drop('Loan_Status_Y', axis=1)
test = test.drop('Loan_Status_N', axis=1)
test = test.fillna(1)
test.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,5720,0.0,110.0,360.0,1.0,0,1,0,1,1,0,0,0,1,0,1,0,0,0,1
1,3076,1500.0,126.0,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1
2,5000,1800.0,208.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
3,2340,2546.0,100.0,360.0,1.0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1
4,3276,0.0,78.0,360.0,1.0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1


In [6]:
print(len(df_train), len(df_test), len(train), len(test))

614 367 529 367


In [7]:
train_train, train_test = train_test_split(
    train,
    test_size=.3,
    random_state=1,
)

In [8]:
X_train = train_train.drop('Loan_Status', axis=1)
Y_train = train_train['Loan_Status']
X_test = train_test.drop('Loan_Status', axis=1)
Y_test = train_test['Loan_Status']
X_train.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
332,2833,0.0,71.0,360.0,1.0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,1
399,1500,1800.0,103.0,360.0,0.0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0
256,6045,0.0,115.0,360.0,0.0,0,1,1,0,1,0,0,0,0,1,1,0,1,0,0
116,3167,2283.0,154.0,360.0,1.0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0
148,10000,1666.0,225.0,360.0,1.0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0


In [9]:
Y_train.head()

332    Y
399    N
256    N
116    Y
148    N
Name: Loan_Status, dtype: object

In [10]:
if do_pca == True:
    pca = PCA(
        n_components=2,
        random_state=1,
    )
    X_train = pca.fit_transform(X_train)
    X_test = pca.fit_transform(X_test)
    test= pca.fit_transform(test)

In [11]:
models = {
    'decision_tree': tree.DecisionTreeClassifier(random_state=1),
    'logistic': LogisticRegression(
        solver='newton-cg',
        penalty='l2',
    ),
    'gboost': ensemble.GradientBoostingClassifier(
        learning_rate=.01,
        random_state=1,
    ),
    'adaboost': ensemble.AdaBoostClassifier(),
    'rf': ensemble.RandomForestClassifier(
        #criterion='gini',
    ),
}
models['bagging'] = ensemble.BaggingClassifier(
    base_estimator=models['gboost'],
    n_estimators=10,
    random_state=1,
)

In [12]:
rows = []
best_model = None
best_accuracy = None
for name, model in models.items():
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_pred, Y_test)
    row = [name, accuracy]
    rows.append(row)
    if best_model == None:
        best_model = model
        best_accuracy = accuracy
    elif accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
df_accuracy = pd.DataFrame(rows, columns=['model', 'accuracy'])
df_accuracy

,model,accuracy
0,decision_tree,0.672956
1,logistic,0.811321
2,gboost,0.836478
3,adaboost,0.742138
4,rf,0.817610
5,bagging,0.842767


In [13]:
predicted = best_model.predict(test)
output = pd.read_csv('../Datasets/loan-prediction/test.csv', usecols=['Loan_ID'])
output['Loan_Status'] = predicted
best_model

BaggingClassifier(base_estimator=GradientBoostingClassifier(learning_rate=0.01,
                                                            random_state=1),
                  random_state=1)

In [14]:
output.to_csv('submission.csv', index=False)